# Load packages and settings

In [1]:
import os
os.system('clear')

import numpy as np
import pandas as pd
from datetime import datetime

from termcolor import colored
from SecretColors.palette import Palette
material = Palette('material', color_mode = 'hexa')

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates
mpl.rcParams['font.family'] = 'SF Compact Text'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams['figure.titlesize'] = 'large'
mpl.rcParams['figure.titleweight'] = 'semibold'

print('')
print(colored('...','white'))
print('')
print(colored(f'Finished successfully', 'green'))
print('')
print(colored('...','white'))
print('')


...

Finished successfully

...



# Convert DA (raw) to DA (df)

In [2]:
yyears = ['2015', '2016', '2017', '2018']

import sys
import calendar

flist = [f for f in sorted(os.listdir("./raw data/DA/")) if not (f.startswith('.') or f.startswith('~'))]

print('')
print(colored('...','white'))
print('')
print(colored(f'Files in directory: {flist}', 'blue'))
print('')

df_da2 = pd.DataFrame()

big_df = pd.DataFrame()
date = []
mcp = []

dst_switch_dates = []

for filename in flist:

    y = filename[24:28]

    if y in yyears:
    
        for month in [3, 10]:
            last_sunday = max(week[-1] for week in calendar.monthcalendar(int(y), month))
            dst_switch_dates.append(datetime.strptime(f'{y}-{calendar.month_abbr[month]}-{last_sunday}','%Y-%b-%d').date())

        print(colored('...','white'))
        print('')
        print(colored(f'Currently processing file: {filename}', 'blue'))
        print(colored(f'Currently processing year: {y}', 'blue'))
        print('')

        df_da = pd.read_excel(f"./raw data/DA/{filename}", sheet_name = "prices_2")
        for index, row in df_da.iterrows():

            for t in range(1, 25):
                t2 = t - 1
                if t <= 9:
                    t = f'0{t}'
                # df2['Date'] = pd.to_datetime(df2['Date'])
                # print(type(row['PRICES'].strftime("%Y-%m-%d") + ' ' + f'{t}' + ':00:00'))
                # df2 = df2.append(row['PRICES'].strftime("%Y-%m-%d") + ' ' + f'{t}' + ':00:00', row[f'Hour {t}'])

                date.append(row['PRICES'].strftime("%Y-%m-%d") + ' ' + f'{t2}' + ':00:00')
                mcp.append(row[f'Hour {t}'])

                # df2['MCP'] = df2['MCP'].append(row[f'Hour {t}'])
                # pd.to_datetime(df_group3['Date'], format='(%Y, %m, %d, %H)')
                
                # df2['Date2'] = pd.to_datetime(df2['Date'], format='%Y-%m-%d %H')

                # pd.to_datetime(df_group3['Date'], format='(%Y, %m, %d, %H)')
                
                # df2['Date'] = pd.to_datetime(df2, format='(%Y, %m, %d, %H)')
        
        import pytz

    df_da2 = pd.DataFrame(date)
    df_da2.columns = ['Date']
    df_da2['MCP'] = pd.DataFrame(mcp)
    df_da2 = df_da2.set_index(pd.DatetimeIndex(df_da2['Date']))
    df_da2 = df_da2.drop(['Date'], axis=1)

    for d in dst_switch_dates:
        df_da2 = df_da2[df_da2.index.date != d]

    # df_da2 = df_da2.tz_localize('Europe/Amsterdam')
    # df_da2 = df_da2.tz_convert('UTC')

    df_da2 = df_da2.tz_localize('UTC')

df_da2.index.names = ['Timestamp']

print(colored('...','white'))
print('')
print(colored(f'I removed the following dates because of DST {dst_switch_dates}', 'magenta'))
print('')

print(colored('...','white'))
print('')
print(colored(f'(DA) Grouped by hour', 'blue'))
print('')

print(colored('...','white'))
print('')
display(df_da2)
print('')

print(colored('...','white'))
print('')
print(colored(f'Finished successfully', 'green'))
print('')
print(colored('...','white'))
print('')


...

Files in directory: ['DAM - Historical data - 2015.xls', 'DAM - Historical data - 2016.xls', 'DAM - Historical data - 2017.xls', 'DAM - Historical data - 2018.xls']

...

Currently processing file: DAM - Historical data - 2015.xls
Currently processing year: 2015

...

Currently processing file: DAM - Historical data - 2016.xls
Currently processing year: 2016

...

Currently processing file: DAM - Historical data - 2017.xls
Currently processing year: 2017

...

Currently processing file: DAM - Historical data - 2018.xls
Currently processing year: 2018

...

I removed the following dates because of DST [datetime.date(2015, 3, 29), datetime.date(2015, 10, 25), datetime.date(2016, 3, 27), datetime.date(2016, 10, 30), datetime.date(2017, 3, 26), datetime.date(2017, 10, 29), datetime.date(2018, 3, 25), datetime.date(2018, 10, 28)]

...

(DA) Grouped by hour

...



,MCP
Timestamp,
2015-01-01 00:00:00+00:00,38.50
2015-01-01 01:00:00+00:00,38.22
2015-01-01 02:00:00+00:00,35.60
2015-01-01 03:00:00+00:00,33.00
2015-01-01 04:00:00+00:00,27.41
...,...
2018-12-31 19:00:00+00:00,58.28
2018-12-31 20:00:00+00:00,50.01
2018-12-31 21:00:00+00:00,47.48



...

Finished successfully

...



In [3]:
df_da3 = pd.DataFrame(columns = ['MCP'])

df_da3['Timestamp'] = pd.date_range(f'{yyears[0]}-01-01 00:00:00', f'{yyears[-1]}-12-31 23:00:00', freq = 'h', tz = 'UTC')

df_da3 = df_da3.set_index('Timestamp')

df_da3['MCP'] = df_da2['MCP']

df_da3.to_pickle(f"./data/DA/DA.pkl")

display(df_da3)

,MCP
Timestamp,
2015-01-01 00:00:00+00:00,38.50
2015-01-01 01:00:00+00:00,38.22
2015-01-01 02:00:00+00:00,35.60
2015-01-01 03:00:00+00:00,33.00
2015-01-01 04:00:00+00:00,27.41
...,...
2018-12-31 19:00:00+00:00,58.28
2018-12-31 20:00:00+00:00,50.01
2018-12-31 21:00:00+00:00,47.48


In [6]:
df_da3[df_da3['MCP'].isna()]

,MCP
Timestamp,
2015-03-29 00:00:00+00:00,NaN
2015-03-29 01:00:00+00:00,NaN
2015-03-29 02:00:00+00:00,NaN
2015-03-29 03:00:00+00:00,NaN
2015-03-29 04:00:00+00:00,NaN
...,...
2018-10-28 19:00:00+00:00,NaN
2018-10-28 20:00:00+00:00,NaN
2018-10-28 21:00:00+00:00,NaN
